In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv(find_dotenv())
db_name = os.environ.get('db_name')
db_user = os.environ.get('db_user')
db_pwd = os.environ.get('db_pwd')
db_host = os.environ.get('db_host')
db_port = os.environ.get('endpoint', 5432)

In [3]:
# Ensure db_port is an integer
db_port = int(db_port)

# Create SQLAlchemy engine for PostgreSQL
try:
    engine = create_engine(f'postgresql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}')
    print("Database connection established.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    raise

# Define function to execute SQL and load data
def create_table_and_load_data(engine, create_table_query, csv_file_path, table_name):
    try:
        # Execute the create table command
        with engine.connect() as connection:
            connection.execute(text(create_table_query))
            print(f"Table {table_name} created successfully or already exists.")

        # Load data into the table
        df = pd.read_csv(csv_file_path)
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"Data from {csv_file_path} loaded successfully into {table_name}.")

    except Exception as e:
        print(f"Error processing table {table_name}: {e}")

Database connection established.


In [4]:
# 1. Customers table
create_table_query1 = """
CREATE TABLE IF NOT EXISTS Customers (
    CustomerKey SERIAL PRIMARY KEY,
    Gender VARCHAR(10),
    Name VARCHAR(255),
    City VARCHAR(255),
    StateCode VARCHAR(255),
    State VARCHAR(255),
    ZipCode VARCHAR(20),
    Country VARCHAR(100),
    Continent VARCHAR(100),
    Birthday DATE,
    Age INT,
    AgeRange VARCHAR(20)
)
"""
create_table_and_load_data(engine, create_table_query1, "cleaned_datasets/Customers.csv", "Customers")

# 2. ExchangeRates table
create_table_query2 = """
CREATE TABLE IF NOT EXISTS ExchangeRates (
    Date DATE,
    Currency VARCHAR(3),
    Exchange FLOAT,
    PRIMARY KEY (Date, Currency)
)
"""
create_table_and_load_data(engine, create_table_query2, "cleaned_datasets/Exchange_Rates.csv", "ExchangeRates")

# 3. Products table
create_table_query3 = """
CREATE TABLE IF NOT EXISTS Products (
    ProductKey SERIAL PRIMARY KEY,
    ProductName VARCHAR(255),
    Brand VARCHAR(100),
    Color VARCHAR(50),
    UnitCostUSD FLOAT,
    UnitPriceUSD FLOAT,
    SubcategoryKey INT,
    Subcategory VARCHAR(255),
    CategoryKey INT,
    Category VARCHAR(255)
)
"""
create_table_and_load_data(engine, create_table_query3, "cleaned_datasets/Products.csv", "Products")

# 4. Sales table
create_table_query4 = """
CREATE TABLE IF NOT EXISTS Sales (
    OrderNumber INT,
    LineItem INT,
    OrderDate DATE,
    DeliveryDate DATE,
    CustomerKey INT,
    StoreKey INT,
    ProductKey INT,
    Quantity INT,
    CurrencyCode VARCHAR(10),
    PRIMARY KEY (OrderNumber, LineItem)
)
"""
create_table_and_load_data(engine, create_table_query4, "cleaned_datasets/Sales.csv", "Sales")

# 5. Stores table
create_table_query5 = """
CREATE TABLE IF NOT EXISTS Stores (
    StoreKey SERIAL PRIMARY KEY,
    Country VARCHAR(100),
    State VARCHAR(100),
    SquareMeters FLOAT,
    OpenDate DATE
)
"""
create_table_and_load_data(engine, create_table_query5, "cleaned_datasets/Stores.csv", "Stores")

Table Customers created successfully or already exists.
Data from cleaned_datasets/Customers.csv loaded successfully into Customers.
Table ExchangeRates created successfully or already exists.
Data from cleaned_datasets/Exchange_Rates.csv loaded successfully into ExchangeRates.
Table Products created successfully or already exists.
Data from cleaned_datasets/Products.csv loaded successfully into Products.
Table Sales created successfully or already exists.
Data from cleaned_datasets/Sales.csv loaded successfully into Sales.
Table Stores created successfully or already exists.
Data from cleaned_datasets/Stores.csv loaded successfully into Stores.
